In [ ]:
#!pip install kaggle

In [5]:
import kaggle
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0




  0%|          | 0.00/200k [00:00<?, ?B/s]
100%|##########| 200k/200k [00:00<00:00, 417kB/s]
100%|##########| 200k/200k [00:00<00:00, 416kB/s]


In [15]:
# extract data from zip file
import zipfile
zip_ref = zipfile.ZipFile('orders.csv.zip')
zip_ref.extractall() # to extract all files to dir
zip_ref.close()

In [27]:
# read data from file and replace naan values
import pandas as pd

df = pd.read_csv('orders.csv', na_values=['Not Available','unknown'])
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [33]:
# standarise class names: all lowercase, no space
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'cost_price', 'list_price', 'quantity',
       'discount_percent'],
      dtype='object')

In [61]:
#df['discount'] = df['list_price']*df['discount_percent'] / 100
#df['sale_price'] = df['list_price']-df['discount']
df['profit'] = df['sale_price']-df['cost_price']
df.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260.0,2,2,5.0,254.7,14.7
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,728.0,3,3,20.6,707.5,107.5
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10.0,2,5,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,959.0,5,2,18.4,940.4,160.4
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20.0,2,5,0.9,19.0,-1.0


In [69]:
# convert order_date into datetime datatype
df['order_date'] = pd.to_datetime(df['order_date'], format="%Y-%m-%d")

In [81]:
# drop unnecessary columns
df.drop(columns=['list_price', 'cost_price', 'discount_percent'], inplace=True)

In [1]:
#load data into sql server
from sqlalchemy import create_engine
server = 'DANYA'
database = 'master'
driver = 'ODBC Driver 17 for SQL Server'
database_con = f'mssql://@{server}/{database}?driver={driver}'

engine = create_engine(database_con)
con = engine.connect()

In [ ]:
df.to_sql('df_orders', con=con, index=False, if_exists='append')